In [19]:
# pip install upgini & catboost
from os.path import exists
import pandas as pd

In [20]:
# Import data
df_path = "train.csv.zip" if exists("train.csv.zip") else "https://github.com/upgini/upgini/raw/main/notebooks/train.csv.zip"
df = pd.read_csv(df_path)
df = df.sample(n=19000, random_state=0)  # random sample of 19k datapts

In [21]:
# Data processing
# Change store & item as strings
df["store"] = df["store"].astype(str)
df["item"] = df["item"].astype(str)

# Convert date into pandas datetime
df["date"] = pd.to_datetime(df["date"])

df.sort_values("date", inplace=True)
df.reset_index(inplace=True, drop=True)
df.head()

,date,store,item,sales
0,2013-01-01,7,5,5
1,2013-01-01,4,9,19
2,2013-01-01,1,33,37
3,2013-01-01,3,41,14
4,2013-01-01,5,24,26


In [22]:
# Split train and test data
# Train: 2013-2016
# Test: 2017
train = df[df["date"] < "2017-01-01"]
test = df[df["date"] >= "2017-01-01"]

In [23]:
# Features and labels
train_features = train.drop(columns=["sales"])  # every column but sales
train_target = train["sales"]  # new sales column
test_features = test.drop(columns=["sales"])
test_target = test["sales"]

Enrich Features (Upgini)

In [24]:
import os
os.environ["UPGINI_API_KEY"] = "vxtqIeggiIWPf_IgqkIvRgmMu0wVqVaWBx6I7_VOljg"

from upgini import FeaturesEnricher, SearchKey
from upgini.metadata import CVType

enricher = FeaturesEnricher(api_key=os.environ["UPGINI_API_KEY"],
    search_keys = {
        "date": SearchKey.DATE,
    },
    cv = CVType.time_series
)
enricher.fit(train_features,
             train_target,
             eval_set=[(test_features, test_target)])

[============================================================] 100% Finished

WARNING #1: Search started with DATE search key only
Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IP to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history


Detected task type: ModelTaskType.REGRESSION. Reason: date search key is present, treating as regression
You can set task type manually with argument `model_task_type` of FeaturesEnricher constructor if task type detected incorrectly

WARNING #2: Your training sample is unstable in number of rows per date. It is recommended to redesign the training sample



Column name,Status,Errors
target,All valid,-
date,All valid,-




Running search request, search_id=2ff4ffb1-5907-4561-a996-281a8509a824
\


f_economic_date_cbpol_umap_6_aa0352de,10.0407,100.0000,"1.0587, 6.9596, 1.0408",Upgini,World economic indicators,Daily
f_autofe_groupbythenmean_d5f4f809f4,8.4062,100.0000,"0.0981, 0.0598, 0.0905","Upgini,Training dataset","AutoFE: feature from Markets data, grouped by feature from training dataset",Daily
f_autofe_groupbythenmedian_64b28257d7,7.9372,100.0000,"1.0005, 1.0012, 1.0011","Upgini,Training dataset","AutoFE: feature from Markets data, grouped by feature from training dataset",Daily
f_autofe_groupbythenrank_d595da62f7,3.5301,100.0000,"0.3525, 0.5, 0.2133","Upgini,Training dataset","AutoFE: feature from Calendar data, grouped by feature from training dataset",Daily
f_autofe_roll_3d_min_1a2ad6d8bb,2.2199,100.0000,"0.3253, -0.2795, -0.3817","Training dataset,Upgini","AutoFE: features from Training dataset,Calendar data",Daily
f_financial_date_crude_oil_7d_to_1y_c3e0ad17,2.1258,100.0000,"1.0001, 1.0769, 1.0154",Upgini,Markets data,Daily
f_autofe_roll_3d_median_8e12f1e1e9,1.6745,100.0000,"0.309, -0.263, -0.3496","Training dataset,Upgini","AutoFE: features from Training dataset,Calendar data",Daily
f_events_date_year_cos1_9014a856,1.4801,100.0000,"0.3253, -0.263, -0.3496",Upgini,Calendar data,Daily
store,1.1122,100.0000,"5, 10, 7",,,
f_autofe_groupbythenrank_110e5b8df1,0.8530,100.0000,"0.0628, 0.2215, 0.7964","Upgini,Training dataset","AutoFE: feature from Calendar data, grouped by feature from training dataset",Daily
f_autofe_groupbythenrank_3729e8053b,0.7708,100.0000,"0.3674, 0.3224, 0.3571","Upgini,Training dataset","AutoFE: feature from Calendar data, grouped by feature from training dataset",Daily


"Upgini,Training dataset","AutoFE: feature from Markets data, grouped by feature from training dataset",16.3434,2
Upgini,World economic indicators,10.0407,1
"Upgini,Training dataset","AutoFE: feature from Calendar data, grouped by feature from training dataset",5.1539,3
"Training dataset,Upgini","AutoFE: features from Training dataset,Calendar data",3.8944,2
Upgini,Calendar data,2.8988,4
Upgini,Markets data,2.1258,1
Training dataset,AutoFE: features from Training dataset,0.5509,2


"Markets data, grouped by feature from training dataset",f_autofe_groupbythenmean_d5f4f809f4,f_financial_date_vix_gap_b64bd2b9,store_824d80,GroupByThenMean
"Markets data, grouped by feature from training dataset",f_autofe_groupbythenmedian_64b28257d7,f_financial_date_usd_eur_1d_to_7d_d8b89b5d,item_4a33ea,GroupByThenMedian
"Calendar data, grouped by feature from training dataset",f_autofe_groupbythenrank_d595da62f7,f_events_date_week_cos3_7525fe31,store_824d80,GroupByThenRank
"Training dataset,Calendar data",f_autofe_roll_3d_min_1a2ad6d8bb,f_events_date_year_cos1_9014a856,,roll_3d_min
"Training dataset,Calendar data",f_autofe_roll_3d_median_8e12f1e1e9,f_events_date_year_cos1_9014a856,,roll_3d_median
"Calendar data, grouped by feature from training dataset",f_autofe_groupbythenrank_110e5b8df1,f_events_date_week_cos1_f6a8c1fc,store_824d80,GroupByThenRank
"Calendar data, grouped by feature from training dataset",f_autofe_groupbythenrank_3729e8053b,f_events_date_week_cos1_f6a8c1fc,item_4a33ea,GroupByThenRank
Training dataset,f_autofe_roll_3d_max_946bf6086b,target,,roll_3d_max
Training dataset,f_autofe_lag_7d_4a1d9c9dff,target,,lag_7d


We detected 48 outliers in your sample.
Examples of outliers with maximum value of target:
33    205
17    196
12    187
Name: target, dtype: int64
Outliers will be excluded during the metrics calculation.
Calculating accuracy uplift after enrichment...
y distributions from the training sample and eval_set differ according to the Kolmogorov-Smirnov test,
which makes metrics between the train and eval_set incomparable.


Train,9418,53.3352,0.324 ± 0.109,0.234 ± 0.085,0.0900,27.9%
Eval 1,3764,58.5994,0.278 ± 0.009,0.208 ± 0.022,0.0700,25.0%


In [25]:
from catboost import CatBoostRegressor
from catboost.utils import eval_metric

model = CatBoostRegressor(verbose=False, allow_writing_files=False, random_state=0)

enricher.calculate_metrics(
    train_features, train_target,
    eval_set = [(test_features, test_target)],
    estimator = model,
    scoring = "mean_absolute_percentage_error"
)

Calculating accuracy uplift after enrichment...
y distributions from the training sample and eval_set differ according to the Kolmogorov-Smirnov test,
which makes metrics between the train and eval_set incomparable.


,Dataset type,Rows,Mean target,Baseline MAPE,Enriched MAPE,"Uplift, abs","Uplift, %"
0,Train,9418,53.3352,0.288 ± 0.096,0.207 ± 0.100,0.081,28.2%
1,Eval 1,3764,58.5994,0.247 ± 0.008,0.184 ± 0.036,0.063,25.6%


In [26]:
# Train model (Upgini free user limit: 1000 rows)
# Keep index to align y properly
train_idx = train_features.sample(n=600, random_state=1).index
test_idx = test_features.sample(n=400, random_state=1).index

train_subset = train_features.loc[train_idx].copy()
test_subset = test_features.loc[test_idx].copy()

# Add features into existing dataset
y_train_subset = train_target.loc[train_idx]

# Fit+transform on train (with y)
enriched_train_features = enricher.fit_transform(
    train_subset,
    y=y_train_subset,
    keep_input=True)

# Transform test (w/o y)
enriched_test_features = enricher.transform(
    test_subset,
    y=test_target.loc[test_idx],
    keep_input=True)

enriched_train_features.head()

[============================================================] 100% Finished

WARNING #1: Search started with DATE search key only
Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IP to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history

WARNING #2: Your training sample is unstable in number of rows per date. It is recommended to redesign the training sample



Column name,Status,Errors
target,All valid,-
date,All valid,-




Running search request, search_id=abf36315-1a60-44c3-af8a-76c94c6bb4f6
\


f_autofe_roll_2d_q25_00488fe847,9.5751,100.0000,"96.0, 72.25, 19.0",Training dataset,AutoFE: features from Training dataset,
f_autofe_roll_2d_min_4199c37ccb,6.8539,100.0000,"46.0, 143.0, 19.0",Training dataset,AutoFE: features from Training dataset,
f_autofe_delta_1d_23f8ab35be,4.0054,31.7003,"6.0, 30.0, -20.0",Training dataset,AutoFE: features from Training dataset,


...

[============================================================] 100% Finished

You use Trial access to Upgini data enrichment. Limit for Trial: 1000 rows. You have already enriched: 600 rows.
WARNING #1: Search started with DATE search key only
Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IP to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history



Column name,Status,Errors
date,All valid,-




Running transform request, id=53c38c2d-04d8-4199-8a49-37cd047a440f
\
Retrieving selected features from data sources...


,date,f_autofe_delta_1d_23f8ab35be,f_autofe_roll_2d_min_4199c37ccb,f_autofe_roll_2d_q25_00488fe847
3176,2013-10-25,NaN,75.0,75.0
8207,2015-02-21,NaN,31.0,31.0
3152,2013-10-23,NaN,16.0,16.0
6579,2014-09-17,-8.0,9.0,11.0
5558,2014-06-12,NaN,34.0,34.0


In [27]:
# Train and evaluate model on original dataset
model.fit(train_features, train_target)
preds = model.predict(test_features)

# Symmetric Mean Absolute % Error between test_target vs model prediction
eval_metric(test_target.values, preds, "SMAPE") 

[37.65141857448004]

In [28]:
from catboost import Pool

# y aligned to enriched X
y_enriched_train = train_target.loc[enriched_train_features.index]
y_enriched_test  = test_target.loc[enriched_test_features.index]

# CatBoost Pools -> avoid type/shape ambiguity
train_pool = Pool(enriched_train_features, y_enriched_train)
test_pool  = Pool(enriched_test_features,  y_enriched_test)

# Train and evaluate model on enriched dataset
model.fit(train_pool)
enriched_preds = model.predict(test_pool)
eval_metric(y_enriched_test.values, enriched_preds, "SMAPE") 

[28.16240309825206]

Forecasting - recursive 3-month forecast

In [30]:
import numpy as np
from collections import deque

df = df.copy()
df["store"] = df["store"].astype(str)
df["item"]  = df["item"].astype(str)
df["date"]  = pd.to_datetime(df["date"])
df = df.sort_values(["store","item","date"]).reset_index(drop=True)

cutoff = pd.Timestamp("2017-01-01")
H=90  # forecast range (days)
lags = [1,7,28]  # configurable lags
rolls = [7,28]  # rolling-window means (per group)

In [32]:
# Feature Engineering
def add_date_features(frame: pd.DataFrame) -> pd.DataFrame:
    f = frame.copy()
    f["dow"]        = f["date"].dt.dayofweek.astype(np.int16)     # 0=Mon
    f["weekofyear"] = f["date"].dt.isocalendar().week.astype(int)
    f["month"]      = f["date"].dt.month.astype(np.int8)
    f["day"]        = f["date"].dt.day.astype(np.int8)
    f["is_weekend"] = (f["dow"] >= 5).astype(np.int8)
    return f

def add_group_lags(frame: pd.DataFrame) -> pd.DataFrame:
    f = frame.copy()
    f = f.sort_values(["store","item","date"])
    # per (store,item)
    f_list = []
    for (_, _), g in f.groupby(["store","item"], sort=False):
        g = g.copy()
        for lag in lags:
            g[f"lag_{lag}"] = g["sales"].shift(lag)
        for w in rolls:
            g[f"rmean_{w}"] = g["sales"].shift(1).rolling(w).mean()
        f_list.append(g)
    f = pd.concat(f_list, ignore_index=True)
    return f

# Build features (only once, using actual history)
feat_df = add_date_features(df)
feat_df = add_group_lags(feat_df)

# Drop rows where lags aren't available yet
min_lag = max(max(lags), max(rolls, default=1))
feat_df = feat_df[feat_df.groupby(["store","item"]).cumcount() >= min_lag].reset_index(drop=True)


In [33]:
# Train/test split
train_mask = feat_df["date"] < cutoff
train_df = feat_df[train_mask].copy()
test_df  = feat_df[~train_mask].copy()

features = [
    "store","item","dow","weekofyear","month","day","is_weekend",
    *[f"lag_{k}" for k in lags],
    *[f"rmean_{w}" for w in rolls]
]
target   = "sales"

# Cat features (strings)
cat_features = ["store","item"]

train_pool = Pool(train_df[features], label=train_df[target], cat_features=cat_features)
test_pool  = Pool(test_df[features],  label=test_df[target],  cat_features=cat_features)

In [45]:
# Train model
model = CatBoostRegressor(
    depth=8, learning_rate=0.02, l2_leaf_reg=5, n_estimators=1000,
    loss_function="RMSE", random_state=0, verbose=False, allow_writing_files=False
)
model.fit(train_pool)

In [46]:
# Recursive forecast for next 90 days
last_date = df["date"].max()
future_dates = pd.date_range(last_date + pd.Timedelta(days=1), periods=H, freq="D")

# Build an initial "state" per (store,item): last known history needed to compute lags/rolls
state = {}
for (s, it), g in df.groupby(["store","item"], sort=False):
    g = g.sort_values("date")
    hist = g["sales"].tolist()
    if len(hist) < min_lag + 1:
        # skip short series / pad with NaNs/zeros
        continue
    state[(s, it)] = {
        "buf": deque(hist[-(max(rolls + lags) + 1):], maxlen=max(rolls + lags) + 1)  # store last N sales
    }

# Collect future rows to predict
rows = []
for d in future_dates:
    for (s, it), st in state.items():
        buf = st["buf"]
        # lag features derived from buffer (e.g. buf[-1] is yesterday)
        lag_vals = {f"lag_{k}": (buf[-k] if len(buf) >= k+1 else np.nan) for k in lags}
        roll_vals = {
            f"rmean_{w}": (np.mean(list(buf)[-w-1:-1]) if len(buf) >= w+1 else np.nan)
            for w in rolls
        }
        x = {
            "date": d, "store": s, "item": it,
        }
        x.update(lag_vals)
        x.update(roll_vals)
        rows.append(x)

future_base = pd.DataFrame(rows)
future_base = add_date_features(future_base)

# Same column order as training features
future_X = future_base[features].copy()

# CatBoost Pool for future
future_pool = Pool(future_X, cat_features=cat_features)

# Predict one day at a time and update buffers (recursive)
future_preds = []
idx = 0
for d in future_dates:
    # Slice the block for this date (all store-item combos)
    block = future_X.iloc[idx: idx + len(state)]
    pool_block = Pool(block, cat_features=cat_features)
    yhat = model.predict(pool_block)
    future_preds.extend(yhat)

    # Update buffers with the predictions for next day's lags
    k = 0
    for key in state.keys():
        state[key]["buf"].append(yhat[k])
        k += 1
    idx += len(state)

future_base["forecast_sales"] = future_preds

In [47]:
# Total forecast per day (all stores/items)
daily_forecast = future_base.groupby("date")["forecast_sales"].sum().reset_index()

# Example: per (store,item) on monthly totals
monthly_forecast = (
    future_base.assign(month=lambda x: x["date"].dt.to_period("M").dt.to_timestamp())
               .groupby(["store","item","month"])["forecast_sales"].sum()
               .reset_index()
)

In [48]:
daily_forecast.head()

,date,forecast_sales
0,2018-01-01,17308.580823
1,2018-01-02,18578.759742
2,2018-01-03,19409.734930
3,2018-01-04,20720.185976
4,2018-01-05,21958.381896


Each row represents the total predicted sales across all stores and items per date. <br>

*Analysis:*
* Upward trend - recent historical sales were trending upward

In [49]:
monthly_forecast.head()

,store,item,month,forecast_sales
0,1,1,2018-01-01,612.555319
1,1,1,2018-02-01,596.124023
2,1,1,2018-03-01,776.488861
3,1,10,2018-01-01,1563.663257
4,1,10,2018-02-01,1580.612547


Each row represents the total predicted sales for each individual item per store per month <br>

**Example:**
* Store 1, Item 1: predicted sales of 565 units in Jan, 580 in Feb, 760 in Mar 2018